In [39]:
import cv2
import numpy as np
import requests
import time
import os

verbose = False



In [40]:
# Recibe una imagen como parametro aplica una mascara que binariza los colores rojos.
def detectar_color_rojo(img):
    imagen_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Definir el rango de rojo en el espacio de color HSV
    rojo_bajo = np.array([0, 100, 100])
    rojo_alto = np.array([15, 255, 255])

    rojo_bajo1 = np.array([165, 100, 100])
    rojo_alto1 = np.array([179, 255, 255])

    # Crea una máscara para los píxeles que caen dentro del rango de rojo
    mascara1= cv2.inRange(imagen_hsv, rojo_bajo, rojo_alto)
    mascara2 = cv2.inRange(imagen_hsv, rojo_bajo1, rojo_alto1)

    mascara = cv2.add(mascara1, mascara2)

    return mascara

# Recibe una imagen como parametro aplica una mascara que binariza los colores negros
def detectar_color_negro(img):
    # Convierte la imagen a espacio de color HSV
    imagen_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Definir el rango de rojo en el espacio de color HSV
    negro_bajo = np.array([0, 0, 0])
    negro_alto = np.array([179, 255, 80])

    # Crea una máscara para los píxeles que caen dentro del rango de rojo
    mascara = cv2.inRange(imagen_hsv, negro_bajo, negro_alto)

    return mascara



In [41]:
def encontrar_contornos_pieza(image, mask, area_max=6000, area_min=200):
    retorno = image.copy()
    contorno,_ = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    grid_contours = []   
    for contour in contorno:
        if area_max > cv2.contourArea(contour) > area_min:
            grid_contours.append(contour)
    return len(grid_contours)

def detectar_pieza(casilla1, area_max=6000, area_min=900):
    mascara_negra = detectar_color_negro(casilla1.copy())
    mascara_roja = detectar_color_rojo(casilla1.copy())

    cantidad_contornos = encontrar_contornos_pieza(casilla1, mascara_negra, area_max, area_min)

    if cantidad_contornos > 0:
        if verbose: print('Pieza detectada')
        return 1
    else:
        cantidad_contornos = encontrar_contornos_pieza(casilla1, mascara_roja, area_max, area_min)

        if cantidad_contornos > 0:
            if verbose: print('Pieza detectada')
            return 2
        else:
            if verbose: print('Casillero vacio')
            return 0    

def solicitar_foto(ruta):
    requests.get('http://192.168.0.131/capture')
    print("Imagen capturada, esperando a que sea procesada por el MCU\n")
    time.sleep(7)
    response = requests.get('http://192.168.0.131/saved-photo')
    time.sleep(2)
    if response.status_code == 200:
        with open(ruta, 'wb') as archivo:
            archivo.write(response.content)
        # Rotacion de imagen
        imagen = cv2.imread(ruta)
        angulo_rotacion = 90
        alto, ancho = imagen.shape[:2]
        centro = (ancho // 2, alto // 2)
        matriz_rotacion = cv2.getRotationMatrix2D(centro, angulo_rotacion, 1.0)
        imagen_rotada = cv2.warpAffine(imagen, matriz_rotacion, (ancho, alto))
        cv2.imwrite(ruta, imagen_rotada)
        
        print(f'Imagen descargada como {ruta}\n')
    else:
        print('Error al descargar la imagen')

def recortar_casillero(image, i, j):
    width = image.shape[1]
    height = image.shape[0]
    start_y = int((height/8)*i)
    end_y = int((height/8)*i+(height/8))
    start_x = int((width/8)*j)
    end_x = int((width/8)*j+(width/8))
    casillero = image[start_y:end_y, start_x:end_x].copy()
    return casillero

def leer_recorte_tablero(filename):
    valores = []
    try:
        # Abre el archivo en modo lectura
        with open(filename, 'r') as archivo:
            for linea in archivo:
                valor = int(linea.strip())
                valores.append(valor)
    except FileNotFoundError:
        print(f"El archivo '{filename}' no se encontró.")
    return valores

def mostrar_contornos(img,min,max):
    # Aplicar deteccion de bordes utilizando Canny
    edges = cv2.Canny(img, min, max, apertureSize=3)
    if verbose:
        cv2.imshow('Detector de contornos', edges)
        cv2.waitKey(0)  
        cv2.destroyAllWindows()
    return edges

def rotar_imagen(img):
    # Convertimos la imagen a gris
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).copy()

    if verbose: 
        cv2.imshow('Imagen en blanco y negro', img_gray)
        cv2.waitKey()
        cv2.destroyAllWindows()

    # Aplicar deteccion de bordes utilizando Canny
    edges = mostrar_contornos(img_gray, 150, 300)

    if verbose: 
        cv2.imshow('Contornos detectados', edges)
        cv2.waitKey()
        cv2.destroyAllWindows()

    # Encontrar las lineas presentes en la imagen utilizando la transformada de Hough
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

    # Calcular el angulo promedio solo de las líneas horizontales detectadas
    angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines if abs(line[0][3] - line[0][1]) < abs(line[0][2] - line[0][0])])

    # Rotar la imagen utilizando el ángulo calculado
    (h, w) = img_gray.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle * 180 / np.pi, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    if verbose: 
        cv2.imshow('Imagen rotada', rotated)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    return rotated

config = leer_recorte_tablero('config.log')

x_initial = config[0]
x_release = config[1]
y_initial = config[2]
y_release = config[3]

ruta = os.getcwd() + '/temp.jpg'

solicitar_foto(ruta)

image = cv2.imread(ruta)
image = rotar_imagen(image)
image = image[(y_initial):(y_release), (x_initial):(x_release)].copy()
image = rotar_imagen(image)
cv2.imwrite(ruta, image)

Imagen capturada, esperando a que sea procesada por el MCU

Imagen descargada como /home/tom/universidad/chessboard-cv/temp.jpg



True

In [44]:
matriz_numerica_t1 = np.zeros((8, 8), dtype=int)
for i in range(8):
    for j in range(8):
        casillero = recortar_casillero(image, i,j)
        #areas
        matriz_numerica_t1[i][j] = detectar_pieza(casillero, 6500, 700)

print(matriz_numerica_t1)

[[1 1 1 1 1 1 1 1]
 [1 0 1 0 1 0 1 1]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [2 2 2 0 2 2 2 2]
 [2 2 2 2 2 2 2 2]]


In [43]:
casillero = recortar_casillero(image, 6,0)
detectar_pieza(casillero, 6500, 700)

2